In [1]:
import numpy as np
import pandas as pd

#### **3. DATA CLEANING & PREPARATION**

In this section, we will focus on data preparation and cleaning, a crucial step in ensuring the accuracy and reliability of our analysis. This phase involves several key tasks aimed at transforming raw data into a format that is ready for analysis. Our primary objectives here include:

- Handling Missing Values: We'll address any missing values identified during our data understanding phase

- Correcting Data Types: We will ensure that each column in our dataset is of the appropriate data type. For instance, dates should be in datetime format, and categorical data should be properly encoded.

- Removing Duplicates: Any duplicate records that could skew the analysis will be identified and removed to maintain the uniqueness of our dataset.

- Standardizing Data: We will standardize inconsistent or erroneous entries, such as varying formats in categorical variables or incorrect data entries, to ensure uniformity across the dataset.

In [2]:
# Load Raw dataset
df_olist_raw = pd.read_csv('./dataset/df_olist_raw.csv')

##### **3.1. Correcting Data Types**

In [3]:
# select all columns that have datetime information
time_columns = df_olist_raw.columns[df_olist_raw.columns.str.contains('date|timestamp', case=False)]
time_columns

Index(['order_purchase_timestamp', 'order_delivered_customer_date',
       'order_estimated_delivery_date', 'review_creation_date',
       'review_answer_timestamp'],
      dtype='object')

In [4]:
# mengubah column dengan keterangan waktu kedalam format datetime
df_olist_raw[time_columns] = df_olist_raw[time_columns].apply(pd.to_datetime)

##### **3.2. Typo Correction**

In [5]:
typo_dict = {
    'são paulo': 'sao paulo',
    'sao pauo': 'sao paulo',
    'sao paulop': 'sao paulo',
    'sao paulo sp': 'sao paulo',
    'sao paulo / sao paulo': 'sao paulo',
    'sao paulo - sp': 'sao paulo',
    'sao paluo': 'sao paulo',
    'sp / sp': 'sao paulo',
    'sp': 'sao paulo',
    'scao jose do rio pardo': 'sao jose do rio pardo',
    'sbc/sp': 'sao bernardo do campo',
    'sbc': 'sao bernardo do campo'
}

df_olist_raw['seller_city'] = df_olist_raw['seller_city'].replace(typo_dict)

df_olist_raw['seller_city'].value_counts().head()

seller_city
sao paulo                29489
ibitinga                  8373
curitiba                  3161
santo andre               3149
sao jose do rio preto     2693
Name: count, dtype: int64

##### **3.3. Handling Missing Values**

We will handling the `order_delivered_customer_date`column first since it has the highest missing value. Our analysis perform only focusing on number of succes order. so we will exclude the order with status canceled and unavailable by directly droping the row from `order_status` column.

In [6]:
def summarize_missing_values(df):
    missing_values = df.isna().sum()
    missing_percentage = (missing_values / len(df)) * 100

    missing_info = pd.DataFrame({
        'Missing Values': missing_values,
        'Percentage': [f"{x:.2f}%" for x in missing_percentage]
    })

    missing_info = missing_info[missing_info['Missing Values'] > 0]

    return missing_info

In [7]:
summarize_missing_values(df_olist_raw)

,Missing Values,Percentage
order_approved_at,177,0.15%
order_delivered_customer_date,3421,2.87%
order_item_id,833,0.70%
product_id,833,0.70%
seller_id,833,0.70%
price,833,0.70%
freight_value,833,0.70%
payment_sequential,3,0.00%
payment_type,3,0.00%
payment_installments,3,0.00%


    3.3.1. order_delivered_customer_date Missing Value

In [8]:
# checking the order with status canceled and unavailable 
missing_vals = df_olist_raw[df_olist_raw['order_status'].isin(['canceled','unavailable'])]
missing_vals.head(2)

,customer_id,customer_unique_id,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,...,payment_installments,payment_value,product_category_name,product_category_name_english,seller_city,seller_state,review_score,review_comment_message,review_creation_date,review_answer_timestamp
52,f34a6e874087ec1f0e3dab9fdf659c5d,233896de79986082f1f479f1f85281cb,ituiutaba,MG,6e98de3a85c84ead6689189b825d35b5,canceled,2018-03-15 10:07:02,2018-03-15 10:29:33,NaT,2018-04-09,...,1.0,73.16,NaN,NaN,NaN,NaN,1.0,Muita demora pra entregar e acima de tudo nao ...,2018-04-11,2018-04-11 13:17:25
341,5bfe800011656c0afb81db64519982db,0071f46a072a9ae25bbe4438b15efe9c,aracatuba,SP,df8c077268f7f3baaac0892eb3143642,unavailable,2017-02-01 00:04:17,2017-02-01 00:30:55,NaT,2017-03-10,...,4.0,174.54,NaN,NaN,NaN,NaN,3.0,Gostaria de saber sobre o meu produto pois já ...,2017-03-12,2017-03-14 11:08:50


The `order_status` with status canceled and unavailable show us there is no `product_id` and `seller_id` which indicated this data is invalid and cannot be used for further analysis. we decided to directly drop all this missing value.

In [9]:
# droping the missing value with status canceled and unavailable
df_olist_raw.drop(df_olist_raw[df_olist_raw['order_status'].isin(['canceled', 'unavailable'])].index, inplace=True)

After that we will also check an order with other order_status information for `shipped`, `invoiced`, `processing`, `created` and `approved`. We will see through the gap different between the `order_estimated_delivery_date` and the latest date from the datasets

In [10]:
# find the latest date in the dataset by taking the maximum value from columns with datetime dtype
datetime_columns = df_olist_raw.select_dtypes(include=['datetime64[ns]', 'datetime64[ns, UTC]', 'datetime64']).columns

max_values = {}

for col in datetime_columns:
    max_values[col] = df_olist_raw[col].max()

max_values_df = pd.DataFrame(max_values.items(), columns=['Datetime Column', 'Max Value'])
max_values_df


,Datetime Column,Max Value
0,order_purchase_timestamp,2018-09-03 09:06:57
1,order_delivered_customer_date,2018-10-17 13:22:46
2,order_estimated_delivery_date,2018-10-25 00:00:00
3,review_creation_date,2018-08-31 00:00:00
4,review_answer_timestamp,2018-10-29 12:27:35


In [11]:
# latest date on dataset
last_date = pd.to_datetime('2018-10-25 00:00:00')

non_delivered_df_olist = df_olist_raw[df_olist_raw['order_status'] != 'delivered']

non_delivered_df_olist['time_difference'] = (last_date - non_delivered_df_olist['order_estimated_delivery_date']).dt.days

non_delivered_df_olist[['order_id', 'order_status', 'order_estimated_delivery_date', 'time_difference']].sort_values(by='time_difference', ascending=True).head()



C:\Users\user\AppData\Local\Temp\ipykernel_20208\1245943324.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_delivered_df_olist['time_difference'] = (last_date - non_delivered_df_olist['order_estimated_delivery_date']).dt.days


,order_id,order_status,order_estimated_delivery_date,time_difference
51865,00a99c50fdff7e36262caba33821875a,shipped,2018-09-18,37
64365,b1c978cd633caac1ce86e36b61e8aeb3,shipped,2018-09-06,49
79458,54282e97f61c23b78330c15b154c867d,shipped,2018-09-06,49
72386,5e6f7229aec06e67732033777ee84816,shipped,2018-09-04,51
14231,ef13a698cf88c9e17ccca0261e0b9e7a,shipped,2018-08-30,56


Orders with a status other than 'delivered' have a time difference of more than 30 days between `order_estimated_delivery_date` and the latest date in the dataset. This indicates that these are orders that failed to reach the customer, and no further action was taken by either the seller or the shipper. With minimal information for proper handling, it is decided that all rows with the above information will be dropped from the dataset.


In [12]:
df_olist_raw['order_status'].value_counts()

order_status
delivered     115723
shipped         1256
invoiced         378
processing       376
created            5
approved           3
Name: count, dtype: int64

In [13]:
df_olist_raw.drop(df_olist_raw[df_olist_raw['order_status'].isin(['shipped', 'invoiced','processing','created','approved'])].index, inplace=True)

    3.3.2. product_category_name Missing Value

next for `product_category_name_english` missing values

There’s a slight difference in missing values between the `product_category_name` in Portuguese and English, with the English version missing 25 more entries. This likely means there are a few categories in Portuguese that don’t have an English translation. Let’s check which ones are missing

In [14]:
nan_product_cat = df_olist_raw[df_olist_raw['product_category_name_english'].isna()]
nan_product_cat = nan_product_cat[['order_id', 'product_id', 'product_category_name', 'product_category_name_english', 'order_status']]
display(nan_product_cat['product_category_name'].value_counts(),
       nan_product_cat)

product_category_name
portateis_cozinha_e_preparadores_de_alimentos    14
pc_gamer                                          9
Name: count, dtype: int64

,order_id,product_id,product_category_name,product_category_name_english,order_status
7,1093c8304c7a003280dd34598194913d,124f74f703e88efe001a10bfa718b6ff,NaN,NaN,delivered
183,f51e3c7f79d1e2c076acbb6af1588b95,e1da02256dce4dee2207c36853547c7b,NaN,NaN,delivered
377,144bc17d37931097a63d8dbb87bfd2a9,f9b1795281ce51b1cf39ef6d101ae8ab,NaN,NaN,delivered
450,cb655436a926b3051ca8d308eee56323,e0f33a3329af6716a0bb47fd7a664439,NaN,NaN,delivered
468,a8318f40d6ce7e1367863809d56a63c3,a4d8f727f92014da5dd64116af14634c,NaN,NaN,delivered
...,...,...,...,...,...
118884,a1fa82769a203e30b8faf81cd32e5193,24aba57735be13fd785bc04d1a8812e4,NaN,NaN,delivered
118885,a1fa82769a203e30b8faf81cd32e5193,24aba57735be13fd785bc04d1a8812e4,NaN,NaN,delivered
118886,a1fa82769a203e30b8faf81cd32e5193,24aba57735be13fd785bc04d1a8812e4,NaN,NaN,delivered
118887,a1fa82769a203e30b8faf81cd32e5193,24aba57735be13fd785bc04d1a8812e4,NaN,NaN,delivered


In [15]:
category_mapping = {
    'portateis_cozinha_e_preparadores_de_alimentos': 'portable_kitchen_and_food_preparers',
    'pc_gamer': 'pc_gamer'
}

df_olist_raw['product_category_name_english'].fillna(
    df_olist_raw['product_category_name'].map(category_mapping),
    inplace=True
)

C:\Users\user\AppData\Local\Temp\ipykernel_20208\406725904.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_olist_raw['product_category_name_english'].fillna(


In [16]:
summarize_missing_values(df_olist_raw)

,Missing Values,Percentage
order_approved_at,15,0.01%
order_delivered_customer_date,8,0.01%
payment_sequential,3,0.00%
payment_type,3,0.00%
payment_installments,3,0.00%
payment_value,3,0.00%
product_category_name,1638,1.42%
product_category_name_english,1638,1.42%
review_score,861,0.74%
review_comment_message,67628,58.44%


We found that the missing values in the `product_category_name` column in Portuguese correspond to missing values in the English column. This suggests that these categories are missing translations in both languages. To simplify things, we’ll update the Portuguese product_category_name column with the corresponding English names from the `product_category_name_english` column. We will then drop the `product_category_name_english` column since it is redundant.

In [17]:
df_olist_raw['product_category_name'] = df_olist_raw['product_category_name_english']
df_olist_raw.drop('product_category_name_english', axis=1, inplace=True)

In [18]:
df_olist_raw['product_category_name'].value_counts().reset_index().head()

,product_category_name,count
0,bed_bath_table,11814
1,health_beauty,9816
2,sports_leisure,8791
3,furniture_decor,8643
4,computers_accessories,7963


Now that all category names are in English. This approach will make the dataset more straightforward and easier to work with.

In [19]:
summarize_missing_values(df_olist_raw)

,Missing Values,Percentage
order_approved_at,15,0.01%
order_delivered_customer_date,8,0.01%
payment_sequential,3,0.00%
payment_type,3,0.00%
payment_installments,3,0.00%
payment_value,3,0.00%
product_category_name,1638,1.42%
review_score,861,0.74%
review_comment_message,67628,58.44%
review_creation_date,861,0.74%


Considering that the `product_category_name` is essential for our recommendation system and the missing values represent just 1.44% of the dataset, we believe the best approach is to drop these rows. This ensures that our recommendation system works with complete and relevant data, which is crucial for making accurate recommendations.

In [20]:
df_olist_raw = df_olist_raw.dropna(subset=['product_category_name'])

In [21]:
summarize_missing_values(df_olist_raw)

,Missing Values,Percentage
order_approved_at,14,0.01%
order_delivered_customer_date,8,0.01%
payment_sequential,3,0.00%
payment_type,3,0.00%
payment_installments,3,0.00%
payment_value,3,0.00%
review_score,849,0.74%
review_comment_message,66707,58.47%
review_creation_date,849,0.74%
review_answer_timestamp,849,0.74%


    3.3.3. payments Missing Value

In [22]:
nan_payment = df_olist_raw[df_olist_raw['payment_type'].isna()]
nan_payment

,customer_id,customer_unique_id,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,...,payment_type,payment_installments,payment_value,product_category_name,seller_city,seller_state,review_score,review_comment_message,review_creation_date,review_answer_timestamp
25365,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04,...,NaN,NaN,NaN,health_beauty,curitiba,PR,1.0,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28
25366,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04,...,NaN,NaN,NaN,health_beauty,curitiba,PR,1.0,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28
25367,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04,...,NaN,NaN,NaN,health_beauty,curitiba,PR,1.0,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28


In [23]:
# ensure we only fill missing values for the specific order_id below
order_id_to_fill = 'bfbd0f9bdef84302105ad712db648a6c'

# fill payment_sequential with the mode (most common value) only where it is missing value (NaN)
df_olist_raw.loc[(df_olist_raw['order_id'] == order_id_to_fill) & (df_olist_raw['payment_sequential'].isna()), 'payment_sequential'] = df_olist_raw['payment_sequential'].mode()[0]

# fill payment_installments with the median value only where it is missing value (NaN)
df_olist_raw.loc[(df_olist_raw['order_id'] == order_id_to_fill) & (df_olist_raw['payment_installments'].isna()), 'payment_installments'] = df_olist_raw['payment_installments'].median()

# fill payment_value with the corresponding price of the order_id and only where it is missing value (NaN)
df_olist_raw.loc[(df_olist_raw['order_id'] == order_id_to_fill) & (df_olist_raw['payment_value'].isna()), 'payment_value'] = df_olist_raw['price']

# fill payment_type with the mode only where it is missing value (NaN)
df_olist_raw.loc[(df_olist_raw['order_id'] == order_id_to_fill) & (df_olist_raw['payment_type'].isna()), 'payment_type'] = df_olist_raw['payment_type'].mode()[0]

# make sure it is applied well
df_olist_raw[df_olist_raw['order_id'] == 'bfbd0f9bdef84302105ad712db648a6c']

,customer_id,customer_unique_id,customer_city,customer_state,order_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_customer_date,order_estimated_delivery_date,...,payment_type,payment_installments,payment_value,product_category_name,seller_city,seller_state,review_score,review_comment_message,review_creation_date,review_answer_timestamp
25365,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04,...,credit_card,2.0,44.99,health_beauty,curitiba,PR,1.0,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28
25366,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04,...,credit_card,2.0,44.99,health_beauty,curitiba,PR,1.0,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28
25367,86dc2ffce2dfff336de2f386a786e574,830d5b7aaa3b6f1e9ad63703bec97d23,sao joaquim da barra,SP,bfbd0f9bdef84302105ad712db648a6c,delivered,2016-09-15 12:16:38,2016-09-15 12:16:38,2016-11-09 07:47:38,2016-10-04,...,credit_card,2.0,44.99,health_beauty,curitiba,PR,1.0,nao recebi o produto e nem resposta da empresa,2016-10-06,2016-10-07 18:32:28


In [24]:
summarize_missing_values(df_olist_raw)

,Missing Values,Percentage
order_approved_at,14,0.01%
order_delivered_customer_date,8,0.01%
review_score,849,0.74%
review_comment_message,66707,58.47%
review_creation_date,849,0.74%
review_answer_timestamp,849,0.74%


    3.3.4. order_approved_at Missing Value

To handle missing values in the `order_approved_at` column for orders where the status is delivered. We will Calculate the Average Time Difference: Compute the average time difference between `order_purchase_timestamp` and order_approved_at for rows where `order_approved_at` is not missing.

In [25]:
df_olist_raw['order_purchase_timestamp'] = pd.to_datetime(df_olist_raw['order_purchase_timestamp'])
df_olist_raw['order_approved_at'] = pd.to_datetime(df_olist_raw['order_approved_at'], errors='coerce')

# Calculate time difference where 'order_approved_at' is not missing
valid_times = df_olist_raw.dropna(subset=['order_approved_at'])
valid_times['time_difference'] = valid_times['order_approved_at'] - valid_times['order_purchase_timestamp']
average_time_difference = valid_times['time_difference'].mean()

# Fill missing 'order_approved_at' values
missing_times = df_olist_raw[df_olist_raw['order_approved_at'].isna()]
df_olist_raw.loc[missing_times.index, 'order_approved_at'] = missing_times['order_purchase_timestamp'] + average_time_difference


C:\Users\user\AppData\Local\Temp\ipykernel_20208\3093250187.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  valid_times['time_difference'] = valid_times['order_approved_at'] - valid_times['order_purchase_timestamp']


In [26]:
summarize_missing_values(df_olist_raw)

,Missing Values,Percentage
order_delivered_customer_date,8,0.01%
review_score,849,0.74%
review_comment_message,66707,58.47%
review_creation_date,849,0.74%
review_answer_timestamp,849,0.74%


    3.3.5. order_delivered_customer Missing Value

To handle missing values in the `order_delivered_customer_date` column for orders where the status is delivered, we will:

- Calculate the Average Time Difference: Compute the average time difference between `order_estimated_delivery_date` and `order_delivered_customer_date` for rows where `order_delivered_customer_date` is not missing.

- Fill Missing Values: Use the average time difference calculated in step 1 to estimate `order_delivered_customer_date` by adding it to the `order_estimated_delivery_date` for rows where `order_delivered_customer_date` is missing.

In [27]:
# Ensure the columns are in datetime format
df_olist_raw['order_delivered_customer_date'] = pd.to_datetime(df_olist_raw['order_delivered_customer_date'])
df_olist_raw['order_estimated_delivery_date'] = pd.to_datetime(df_olist_raw['order_estimated_delivery_date'])

# Calculate the average time difference in days directly
average_time_difference = (
    (df_olist_raw['order_delivered_customer_date'] - df_olist_raw['order_estimated_delivery_date'])
    .dropna()
    .dt.total_seconds() / (60 * 60 * 24)
).mean()

In [28]:

# Fill missing values in 'order_delivered_customer_date'
df_olist_raw['order_delivered_customer_date'].fillna(
    (df_olist_raw['order_estimated_delivery_date'] + pd.to_timedelta(average_time_difference, unit='d')).dt.round('S'),
    inplace=True
)

C:\Users\user\AppData\Local\Temp\ipykernel_20208\51069371.py:3: FutureWarning: 'S' is deprecated and will be removed in a future version, please use 's' instead.
  (df_olist_raw['order_estimated_delivery_date'] + pd.to_timedelta(average_time_difference, unit='d')).dt.round('S'),
C:\Users\user\AppData\Local\Temp\ipykernel_20208\51069371.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_olist_raw['order_delivered_customer_date'].fillna(


    3.3.6. review_creation_date Missing Value

In [29]:
summarize_missing_values(df_olist_raw)

,Missing Values,Percentage
review_score,849,0.74%
review_comment_message,66707,58.47%
review_creation_date,849,0.74%
review_answer_timestamp,849,0.74%


To handle missing values in the review_creation_date column for orders where the status is delivered, we will:

- Calculate the Average Time Difference: Compute the average time difference between `order_delivered_customer_date` and `order_purchase_timestamp` for rows where `order_delivered_customer_date` is not missing.

- Fill Missing Values: Use the average time difference calculated in step 1 to `estimate review_creation_date` by adding it to the `order_delivered_customer_date` for rows where `review_creation_date` is missing.

In [30]:
# Calculate time difference between order_delivered_customer_date and order_purchase_timestamp for non-missing values
time_differences = df_olist_raw['order_delivered_customer_date'] - df_olist_raw['order_purchase_timestamp']
average_time_difference = time_differences.mean()

# Fill missing review_creation_date values
missing_reviews = df_olist_raw[df_olist_raw['review_creation_date'].isna()]
df_olist_raw.loc[missing_reviews.index, 'review_creation_date'] = missing_reviews['order_delivered_customer_date'] + average_time_difference

In [31]:
summarize_missing_values(df_olist_raw)

,Missing Values,Percentage
review_score,849,0.74%
review_comment_message,66707,58.47%
review_answer_timestamp,849,0.74%


    3.3.7. review_score Missing Value


In [32]:
# Menghitung median review score per seller
avg_review_score_by_seller = df_olist_raw.groupby('seller_id')['review_score'].median()

# Menggabungkan median review score dengan dataset utama
df_olist_raw = df_olist_raw.merge(avg_review_score_by_seller, on='seller_id', how='left', suffixes=('', '_median'))

# Mengisi nilai review score yang hilang dengan median per seller
df_olist_raw['review_score'] = df_olist_raw['review_score'].fillna(df_olist_raw['review_score_median'])

# Menghapus kolom median setelah digunakan
df_olist_raw.drop(columns=['review_score_median'], inplace=True)

# Mengisi nilai yang masih hilang dengan median keseluruhan dan membulatkannya
df_olist_raw['review_score'] = df_olist_raw['review_score'].fillna(df_olist_raw['review_score'].median()).round()


    3.3.8. review_asnwer Missing Value

In [33]:
# 1. Convert columns to datetime
df_olist_raw['order_delivered_customer_date'] = pd.to_datetime(df_olist_raw['order_delivered_customer_date'])
df_olist_raw['review_answer_timestamp'] = pd.to_datetime(df_olist_raw['review_answer_timestamp'])

#2. Calculate the gap time
gap_time = (df_olist_raw['review_answer_timestamp'] - df_olist_raw['order_delivered_customer_date']).dt.total_seconds() / (60 * 60 * 24)

# 3. Calculate the average gap time in days [ignoring NaN values]
average_gap_time = gap_time.mean()

# 4. Showing the average gap time
average_gap_time

2.9602630509756422

The mean value of 2.99 days represents the average time difference between the order delivery date and the review answer date. This means that, on average, reviews were answered about 3 days after the order was delivered. This average can be used to fill in missing review answer dates by adding approximately 3 days to the order's delivery date.

In [34]:
# 5. Fill missing review_answer_timestamp with the average gap time
df_olist_raw['review_answer_timestamp'] = df_olist_raw['review_answer_timestamp'].fillna(
    df_olist_raw['order_delivered_customer_date'] + pd.to_timedelta(average_gap_time, unit='d')
)

# 6. Round the filled dates to seconds
df_olist_raw['review_answer_timestamp'] = df_olist_raw['review_answer_timestamp'].dt.round('s')


    3.3.9 review_comment Missing Value

**NOTE** : We will not impute or remove missing data in the review comment column because our primary objective is to analyze the existing comments for customer complaints. Since we're focused on identifying key issues in the purchasing process, the analysis will rely on the available data without needing a complete dataset. This approach allows us to capture valuable insights from the comments that are present, even if the column contains missing values

##### **3.4. State Mapping**

In [35]:
state_mapping = {
    'AC': 'Acre',
    'AL': 'Alagoas',
    'AP': 'Amapá',
    'AM': 'Amazonas',
    'BA': 'Bahia',
    'CE': 'Ceará',
    'DF': 'Distrito Federal',
    'ES': 'Espírito Santo',
    'GO': 'Goiás',
    'MA': 'Maranhão',
    'MT': 'Mato Grosso',
    'MS': 'Mato Grosso do Sul',
    'MG': 'Minas Gerais',
    'PA': 'Pará',
    'PB': 'Paraíba',
    'PR': 'Paraná',
    'PE': 'Pernambuco',
    'PI': 'Piauí',
    'RJ': 'Rio de Janeiro',
    'RN': 'Rio Grande do Norte',
    'RS': 'Rio Grande do Sul',
    'RO': 'Rondônia',
    'RR': 'Roraima',
    'SC': 'Santa Catarina',
    'SP': 'São Paulo',
    'SE': 'Sergipe',
    'TO': 'Tocantins'
}

# Update the 'customer_state' column using the state_mapping dictionary
df_olist_raw['customer_state'] = df_olist_raw['customer_state'].map(state_mapping)


##### **3.5. Factorization**

In this process, we did the factorization for columns like customer_unique_id and seller_id to make it easier when carrying out data analysis as well during visualization. This conversion of the ID already in text format into numbers is a factorization operation which will be helpful during machine learning algorithms to deal with it. In addition, the factorization assists in better storage & processing efficiency and take away some of complexity when interpreting relationships between variables present in data. So factorization step is an essential part to handle data analysis properly which generate effective models.

    3.5.1 Before Factorization

In [36]:
df_olist_raw[['customer_unique_id','seller_id','product_id']].describe().T

,count,unique,top,freq
customer_unique_id,114085,92098,9a736b248f67d166d2fbb006bcb877c3,75
seller_id,114085,2914,4a3ca9315b744ce9f8e9374361493884,2116
product_id,114085,31632,aca2eb7d00ea1a7b8ebd4e68314663af,529


    3.5.2 Factorization Process

In [37]:
# 1. Factorize and prefix Customer IDs
df_olist_raw["customer_unique_id"] = pd.factorize(df_olist_raw["customer_unique_id"], sort=True)[0] + 1
df_olist_raw["customer_unique_id"] = df_olist_raw["customer_unique_id"].apply(lambda x: f"C{x}")

#2. Factorize and prefix Seller IDs
df_olist_raw["seller_id"] = pd.factorize(df_olist_raw["seller_id"], sort=True)[0] + 1
df_olist_raw["seller_id"] = df_olist_raw["seller_id"].apply(lambda x: f"S{x}")

# 3. Factorize and prefix Product IDs
df_olist_raw["product_id"] = pd.factorize(df_olist_raw["product_id"], sort=True)[0] + 1
df_olist_raw["product_id"] = df_olist_raw["product_id"].apply(lambda x: f"P{x}")


    3.5.3 After Factorization

In [38]:
df_olist_raw[['customer_unique_id','seller_id','product_id']].describe().T

,count,unique,top,freq
customer_unique_id,114085,92098,C55602,75
seller_id,114085,2914,S846,2116
product_id,114085,31632,P21247,529


the frequenct of unique value each columns remains same, nothing is change

##### **3.6. Saving the clean dataset**

In [39]:
import os

# saving the the raw datasets after join
folder_path = 'dataset'
file_name = 'df_olist_clean.csv'
file_path = os.path.join(folder_path, file_name)
df_olist_raw.to_csv(file_path, index=False)